# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [3]:
df['Embarked'].isna().sum()

2

In [4]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""

# 類別特徵選擇"Embarked"；數值特徵選擇"Fare"
df['Embarked'] = df['Embarked'].fillna('None')
mean_df = df.groupby(['Embarked'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Embarked'])['Fare'].apply(lambda x:x.mode()[0]).reset_index()
median_df = df.groupby(['Embarked'])['Fare'].median().reset_index()
max_df = df.groupby(['Embarked'])['Fare'].max().reset_index()
min_df = df.groupby(['Embarked'])['Fare'].min().reset_index()

temp = pd.merge(mean_df, mode_df, how='left', on='Embarked')
temp = pd.merge(temp, median_df, how='left', on='Embarked')
temp = pd.merge(temp, max_df, how='left', on='Embarked')
temp = pd.merge(temp, min_df, how='left', on='Embarked')

temp.columns = ['Embarked', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 'Fare_min']
temp

,Embarked,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min
0,C,59.954144,7.2292,29.70,512.3292,4.0125
1,None,80.000000,80.0000,80.00,80.0000,80.0000
2,Q,13.276030,7.7500,7.75,90.0000,6.7500
3,S,27.079812,8.0500,13.00,263.0000,0.0000


In [5]:
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df = df.merge(temp, how='left', on='Embarked')
df.drop('Embarked', axis=1, inplace=True)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,27.079812,8.0500,13.0,263.0000,0.0000
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,59.954144,7.2292,29.7,512.3292,4.0125
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,27.079812,8.0500,13.0,263.0000,0.0000
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,27.079812,8.0500,13.0,263.0000,0.0000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,27.079812,8.0500,13.0,263.0000,0.0000


In [7]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

10 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 'Fare_min']



,Pclass,Age,SibSp,Parch,Fare,Fare_mean,Fare_mode,Fare_median,Fare_max,Fare_min
0,3,22.0,1,0,7.2500,27.079812,8.0500,13.0,263.0000,0.0000
1,1,38.0,1,0,71.2833,59.954144,7.2292,29.7,512.3292,4.0125
2,3,26.0,0,0,7.9250,27.079812,8.0500,13.0,263.0000,0.0000
3,1,35.0,1,0,53.1000,27.079812,8.0500,13.0,263.0000,0.0000
4,3,35.0,0,0,8.0500,27.079812,8.0500,13.0,263.0000,0.0000


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

## Ans:
- 有些微改善，但差異不算大。

In [8]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_minus = df.drop(['Fare_mean', 'Fare_mode', 'Fare_median', 'Fare_max', 'Fare_min'], axis=1)
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
scores = cross_val_score(estimator, train_X, train_Y, cv=5)
print('cross_val_score with original features + logistic regression.')
print('scores = ', scores)
print('mean validation scores = ', scores.mean())

cross_val_score with original features + logistic regression.
scores =  [0.62569832 0.68715084 0.70786517 0.74719101 0.75141243]
mean validation scores =  0.7038635542329971


In [9]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
scores = cross_val_score(estimator, train_X, train_Y, cv=5)
print('cross_val_score with new features + logistic regression.')
print('scores = ', scores)
print('mean validation scores = ', scores.mean())

cross_val_score with new features + logistic regression.
scores =  [0.62569832 0.68156425 0.70224719 0.74157303 0.75706215]
mean validation scores =  0.7016289882888318
